# Homework and bake-off: Sentiment analysis

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2022"

In [ ]:
import sys
!{sys.executable} -m spacy download en_core_web_sm

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [Train set](#Train-set)
1. [Dev sets](#Dev-sets)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
    1. [Token-level differences [1 point]](#Token-level-differences-[1-point])
    1. [Training on some of the bakeoff data [1 point]](#Training-on-some-of-the-bakeoff-data-[1-point])
    1. [A more powerful vector-averaging baseline [2 points]](#A-more-powerful-vector-averaging-baseline-[2-points])
    1. [BERT encoding [2 points]](#BERT-encoding-[2-points])
    1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission instructions](#Submission-instructions)

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis using the ternary (positive/negative/neutral) version of the Stanford Sentiment Treebank (SST-3) as well as a new dev/test dataset drawn from restaurant reviews. Our goal in introducing the new dataset is to push you to create a system that performs well in both the movie and restaurant domains.

The homework questions ask you to implement some baseline system, and the bakeoff challenge is to define a system that does well at both the SST-3 test set and the new restaurant test set. Both are ternary tasks, and our central bakeoff score is the mean of the macro-FI scores for the two datasets. This assigns equal weight to all classes and datasets regardless of size.

The SST-3 test set will be used for the bakeoff evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by developing their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

One of our goals for this homework and bakeoff is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Design a new system. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your system, using random train/test splits initially.
1. If you have time, compare your system with others using `sst.compare_models` or `utils.mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). The only constraint we need to place on your system is that it must have a `predict_one` method that can map directly from an example text to a prediction, and it must be able to make predictions without having any information beyond the text. (For example, it can't depend on knowing which task the text comes from.) See [the bakeoff section below](#Bakeoff-[1-point]) for examples of functions that conform to this specification.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [2]:
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch.nn as nn


from torch_rnn_classifier import TorchRNNClassifier
from torch_tree_nn import TorchTreeNN
import sst
import utils

In [3]:
SST_HOME = os.path.join('data', 'sentiment')
GLOVE_HOME = os.path.join('data', 'glove.6B')

## Train set

Our primary train set is the SST-3 train set:

In [4]:
sst_train = sst.train_reader(SST_HOME)

In [5]:
sst_train.shape[0]

8544

This is the train set we will use for all the regular homework questions. You are welcome to bring in new datasets for your original system. You are also free to add `include_subtrees=True`. This is very likely to lead to better systems, but it substantially increases the overall size of the dataset (from 8,544 examples to 159,274), which will in turn substantially increase the time it takes to run experiments.

See [this notebook](sst_01_overview.ipynb) for additional details of this dataset.

## Dev sets

We have two development set. SST3-dev consists of sentences from movie reviews, just like SST-3 train:

In [6]:
sst_dev = sst.dev_reader(SST_HOME)

Our new bakeoff dev set consists of sentences from restaurant reviews:

In [7]:
bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)

In [8]:
bakeoff_dev.sample(3, random_state=1).to_dict(orient='records')

[{'example_id': 57,
  'sentence': 'I would recommend that you make reservations in advance.',
  'label': 'neutral',
  'is_subtree': 0},
 {'example_id': 590,
  'sentence': 'We were welcomed warmly.',
  'label': 'positive',
  'is_subtree': 0},
 {'example_id': 1968,
  'sentence': 'We have been to Oceanaire twice in the last 6 weeks.',
  'label': 'neutral',
  'is_subtree': 0}]

Here is the label distribution:

In [9]:
bakeoff_dev.label.value_counts()

neutral     1019
positive     777
negative     565
Name: label, dtype: int64

The label distribution for the corresponding test set is similar to this.

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models:

In [10]:
def unigrams_phi(text):
    return Counter(text.split())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [11]:
def fit_softmax_classifier(X, y):
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [12]:
softmax_experiment = sst.experiment(
    sst.train_reader(SST_HOME),   # Train on any data you like except SST-3 test!
    unigrams_phi,                 # Free to write your own!
    fit_softmax_classifier,       # Free to write your own!
    assess_dataframes=[sst_dev, bakeoff_dev]) # Free to change this during development!

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.272     0.692     0.391       565
     neutral      0.429     0.113     0.179      1019
    positive      0.409     0.346     0.375       777

    accuracy                          0.328      2361
   macro avg      0.370     0.384     0.315      2361
weighted avg      0.385     0.328     0.294      2361

Mean of macro-F1 scores: 0.416


`softmax_experiment` contains a lot of information that you can use for error analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with `TorchRNNClassifier`.

To featurize examples for an RNN, we can just get the words in order, letting the model take care of mapping them into an embedding space.

In [13]:
def rnn_phi(text):
    return text.split()

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings). See also [torch_model_base.py](torch_model_base.py) for details on the many optimization parameters that `TorchRNNClassifier` accepts.

In [14]:
def fit_rnn_classifier(X, y):
    sst_glove_vocab = utils.get_vocab(X, mincount=2)
    mod = TorchRNNClassifier(
        sst_glove_vocab,
        early_stopping=True)
    mod.fit(X, y)
    return mod

In [15]:
rnn_experiment = sst.experiment(
    sst.train_reader(SST_HOME),
    rnn_phi,
    fit_rnn_classifier,
    vectorize=False,  # For deep learning, use `vectorize=False`.
    assess_dataframes=[sst_dev, bakeoff_dev])

Stopping after epoch 38. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.0902989357709885

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.577     0.645     0.609       428
     neutral      0.295     0.170     0.216       229
    positive      0.607     0.671     0.637       444

    accuracy                          0.557      1101
   macro avg      0.493     0.495     0.488      1101
weighted avg      0.531     0.557     0.539      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.262     0.662     0.376       565
     neutral      0.424     0.161     0.233      1019
    positive      0.367     0.259     0.303       777

    accuracy                          0.313      2361
   macro avg      0.351     0.361     0.304      2361
weighted avg      0.366     0.313     0.290      2361

Mean of macro-F1 scores: 0.396


## Error analysis

This section begins to build an error-analysis framework using the dicts returned by `sst.experiment`. These have the following structure:

```
'model': trained model
'phi': the feature function used
'train_dataset':
   'X': feature matrix
   'y': list of labels
   'vectorizer': DictVectorizer,
   'raw_examples': list of raw inputs, before featurizing   
'assess_datasets': list of datasets, each with the same structure as the value of 'train_dataset'
'predictions': list of lists of predictions on the assessment datasets
'metric': `score_func.__name__`, where `score_func` is an `sst.experiment` argument
'score': the `score_func` score on the each of the assessment dataasets
```
The following function just finds mistakes, and returns a `pd.DataFrame` for easy subsequent processing:

In [ ]:
def find_errors(experiment):
    """Find mistaken predictions.

    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.

    Returns
    -------
    pd.DataFrame

    """
    dfs = []
    for i, dataset in enumerate(experiment['assess_datasets']):
        df = pd.DataFrame({
            'raw_examples': dataset['raw_examples'],
            'predicted': experiment['predictions'][i],
            'gold': dataset['y']})
        df['correct'] = df['predicted'] == df['gold']
        df['dataset'] = i
        dfs.append(df)
    return pd.concat(dfs)

In [ ]:
softmax_analysis = find_errors(softmax_experiment)

In [ ]:
rnn_analysis = find_errors(rnn_experiment)

Here we merge the sotmax and RNN experiments into a single DataFrame:

In [ ]:
analysis = softmax_analysis.merge(
    rnn_analysis, left_on='raw_examples', right_on='raw_examples')

analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [ ]:
# Examples where the softmax model is correct, the RNN is not,
# and the gold label is 'positive'

error_group = analysis[
    (analysis['predicted_x'] == analysis['gold'])
    &
    (analysis['predicted_y'] != analysis['gold'])
    &
    (analysis['gold'] == 'positive')
]

In [ ]:
error_group.shape[0]

In [ ]:
for ex in error_group['raw_examples'].sample(5, random_state=1):
    print("="*70)
    print(ex)

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Token-level differences [1 point]

We can begin to get a sense for how our two dev sets differ by considering the most frequent tokens from each. This question asks you to begin such analysis.

Your task: write a function `get_token_counts` that, given a `pd.DataFrame` in the format of our datasets, tokenizes the example sentences based on whitespace and creates a count distribution over all of the tokens. The function should return a `pd.Series` sorted by frequency; if you create a count dictionary `d`, then `pd.Series(d).sort_values(ascending=False)` will give you what you need.

In [ ]:
def get_token_counts(df):
    ##### YOUR CODE HERE
    count_dict = sum((Counter(s.split()) for s in df['sentence'].values), Counter())
    return pd.Series(count_dict).sort_values(ascending=False)



In [ ]:
def test_get_token_counts(func):
    df = pd.DataFrame([
        {'sentence': 'a a b'},
        {'sentence': 'a b a'},
        {'sentence': 'a a a b.'}])
    result = func(df)
    print(result)
    for token, expected in (('a', 7), ('b', 2), ('b.', 1)):
        actual = result.loc[token]
        assert actual == expected, \
            "For token {}, expected {}; got {}".format(
            token, expected, actual)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_token_counts(get_token_counts)

In [ ]:
get_token_counts(sst_dev)[50:65]

In [ ]:
get_token_counts(bakeoff_dev)[45:60]

As you develop your original system, you might review these results. The two dev sets have different vocabularies and different low-level encoding details that are sure to impact model performance, especially when one considers that the train set is like `sst_dev` in all these respects. For additional discussion, see [this notebook section](sst_01_overview.ipynb#Tokenization).

### Training on some of the bakeoff data [1 point]

We have so far presented the bakeoff dev set as purely for evaluation. Since the train set consists entirely of SST-3 data, this makes the bakeoff split especially challenging. We might be able to reduce the challenging by adding some of the bakeoff dev set to the train set, keeping some of it for evaluation. The current question asks to begin explore the effects of such training.

Your task: write a function `run_mixed_training_experiment`. The function should:

1. Take as inputs (a) a model training wrapper like `fit_softmax_classifier` and (b) an integer `bakeoff_train_size` specifying the number of examples from `bakeoff_dev` that should be included in the train set.
1. Split `bakeoff_dev` so that the first `bakeoff_train_size` examples are in the train set and the rest are used for evaluation.
1. Use `sst.experiment` with the user-supplied model training wrapper, `unigram_phi` as defined above, and a train set that consists of SST-3 train and the train portion of `bakeoff_dev` as defined in step 2. The value of `assess_dataframes` should be a list consisting of the SST-3 dev set and the evaluation portion of `bakeoff_dev` as defined in step 2.
1. Return the return value of `sst.experiment`.

The function `test_run_mixed_training_experiment` will help you iterate to the required design.

In [ ]:
def run_mixed_training_experiment(wrapper_func, bakeoff_train_size):
    ##### YOUR CODE HERE
    bakeoff_train = bakeoff_dev.iloc[:bakeoff_train_size]
    bakeoff_dev_eval = bakeoff_dev.iloc[bakeoff_train_size:]
    mixed_train_df = pd.concat([sst_train, bakeoff_train])
    
    return sst.experiment(
        mixed_train_df,
        unigrams_phi, #but remember they have different tokenization schemes
        wrapper_func,  # For deep learning, use `vectorize=False`.
        assess_dataframes=[sst_dev, bakeoff_dev_eval])


In [ ]:
def test_run_mixed_training_experiment(func):
    bakeoff_train_size = 1000
    experiment = func(fit_softmax_classifier, bakeoff_train_size)

    assess_size = len(experiment['assess_datasets'])
    assert len(experiment['assess_datasets']) == 2, \
        ("The evaluation should be done on two datasets: "
         "SST3 and part of the bakeoff dev set. "
         "You have {} datasets.".format(assess_size))

    bakeoff_test_size = bakeoff_dev.shape[0] - bakeoff_train_size
    expected_eval_examples = bakeoff_test_size + sst_dev.shape[0]
    eval_examples = sum(len(d['raw_examples']) for d in experiment['assess_datasets'])
    assert expected_eval_examples == eval_examples, \
        "Expected {} evaluation examples; got {}".format(
        expected_eval_examples, eval_examples)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_mixed_training_experiment(run_mixed_training_experiment)

### A more powerful vector-averaging baseline [2 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST-3 problem in which each example is modeled as the mean of its GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. To address this question, we want to get set up to run the experiment with a shallow neural classifier. 

Your task: write and submit a model wrapper function around `TorchShallowNeuralClassifier`. This function should implement hyperparameter search according to this specification:

* Set `early_stopping=True` for all experiments.
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh()` and `nn.ReLU()`.
* For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.

See [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples. You are not required to run a full evaluation with this function using `sst.experiment`, but we assume you will want to.

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strong baseline feature representation scheme for SST-3, so we're sort of hoping you feel compelled to try out variations on your own.

In [ ]:
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

def fit_shallow_neural_classifier_with_hyperparameter_search(X, y):
    ##### YOUR CODE HERE
    mod = TorchShallowNeuralClassifier(early_stopping=True, hidden_dim=100)
    param_grid = {
        'hidden_dim': [50, 100, 200],
        'hidden_activation': [nn.Tanh(), nn.ReLU()]}

    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, mod,
        param_grid=param_grid,
        cv=3)
    return bestmod


In [ ]:
glove_lookups = {50: utils.glove2dict(os.path.join(GLOVE_HOME, f'glove.6B.50d.txt')),
                 300: utils.glove2dict(os.path.join(GLOVE_HOME, f'glove.6B.300d.txt'))}


In [ ]:
def glove_phi(text, dims: int = 50, np_func = np.mean, tokenizer=None):
    assert dims in [50, 300], "Glove data only available with vector size 50 or 300."
    glove_lookup = glove_lookups[dims]
    if not tokenizer:
        tokens = text.split()
    else:
        tokens = tokenizer(text)
    embeddings = np.array([glove_lookup[tok] for tok in tokens if tok in glove_lookup])
    if len(embeddings) == 0:
        dim = len(next(iter(glove_lookup.values())))
        feats = np.zeros(dim)
    else:
        feats = np_func(embeddings, axis=0)
    return feats

In [ ]:
glove_50_shallownet = sst.experiment(sst_train, 
               glove_phi,
               fit_shallow_neural_classifier_with_hyperparameter_search,
               vectorize=False,
               assess_dataframes=[sst_dev])


In [ ]:
from functools import partial
glove_phi_300 = partial(glove_phi,dims=300)

glove_300_shallownet = sst.experiment(sst_train, 
               glove_phi_300,
               fit_shallow_neural_classifier_with_hyperparameter_search,
               vectorize=False,
               assess_dataframes=[sst_dev, bakeoff_dev])

### BERT encoding [2 points]

We might hypothesize that encoding our examples with BERT will yield improvements over the GloVe averaging method explored in the previous question, since BERT implements a much more complex and data-driven function for this kind of combination. This question asks you to begin exploring this general hypothesis.

Your task: write a function `hf_cls_phi` that uses Hugging Face functionality to encode individual examples with BERT and returns the final output representation above the [CLS] token.

You are not required to evaluate this feature function, but it is easy to do so with `sst.experiment` and `vectorize=False` (since your feature function directly encodes every example as a vector). Your code should also be a natural basis for even more powerful approaches – for example, it might be even better to pool all the output states rather than using just the first output state. Another option is [fine-tuning](finetuning.ipynb).

In [ ]:
sst_train_vocab = utils.get_vocab(sst_train, mincount=2)

In [ ]:
from transformers import BertModel, BertTokenizer
import vsm

# Instantiate a Bert model and tokenizer based on `bert_weights_name`:
bert_weights_name = 'bert-base-uncased'
##### YOUR CODE HERE
bert_model = BertModel.from_pretrained(bert_weights_name)
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name) # do I need to fix the vocab somehow?

def hf_cls_phi(text):
    # Get the ids. `vsm.hf_encode` will help; be sure to
    # set `add_special_tokens=True`.
    ##### YOUR CODE HERE
    ids = vsm.hf_encode(text, bert_tokenizer, add_special_tokens=True)

    # Get the BERT representations. `vsm.hf_represent` will help:
    ##### YOUR CODE HERE
    reps = vsm.hf_represent(ids, bert_model)
    
    print(reps.shape)
    print(reps[0].shape)

    # Index into `reps` to get the representation above [CLS].
    # The shape of `reps` should be (1, n, 768), where n is the
    # number of tokens. You need the 0th element of the 2nd dim:
    ##### YOUR CODE HERE
    cls_rep = reps[0][0]

    # These conversions should ensure that you can work with the
    # representations flexibly. Feel free to change the variable
    # name:
    return cls_rep.cpu().numpy()

In [ ]:
def test_hf_cls_phi(func):
    rep = func("Just testing!")

    expected_shape = (768,)
    result_shape = rep.shape
    assert rep.shape == (768,), \
        "Expected shape {}; got {}".format(
        expected_shape, result_shape)

    # String conversion to avoid precision errors:
    expected_first_val = str(0.1709)
    result_first_val = "{0:.04f}".format(rep[0])

    assert expected_first_val == result_first_val, \
        ("Unexpected representation values. Expected the "
        "first value to be {}; got {}".format(
            expected_first_val, result_first_val))

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_hf_cls_phi(hf_cls_phi)

Note: encoding all of SST-3 train (no subtrees) takes about 11 minutes on my 2015 iMac, CPU only (32GB).

### Your original system [3 points]

Your task is to develop an original model for the SST-3 problem and our new bakeoff dataset. There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. Our [practical introduction to contextual word representations](finetuning.ipynb) covers pretrained representations and interfaces that are likely to boost the performance of any system.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it. See the course website for additional guidance on how original systems will be evaluated.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development (your best average of macro-F1 scores), just to help us understand how systems performed overall.

<font color='red'>Please review the descriptions in the following comment and follow the instructions.</font>

## Methods

In [13]:
from sklearn.decomposition import TruncatedSVD, PCA, NMF
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier


def fit_pipeline_classifier(X, y, mod):

    pipeline = Pipeline([
        ('scaler', TfidfTransformer()),
        #('dim_reduction', pca),
        # We need this little bridge to go from
        # sparse matrices to dense ones:
        ('densify', utils.DenseTransformer()),
        ('model', mod)])
    pipeline.fit(X, y)
    return pipeline

In [14]:
class TorchDeeperNeuralClassifier(TorchShallowNeuralClassifier):
    def __init__(self, num_hidden_layers=1, **base_kwargs):
        super().__init__(**base_kwargs)
        self.num_hidden_layers = num_hidden_layers
        self.params += ['num_hidden_layers']

    def build_graph(self):
        hiddens = [nn.Linear(self.hidden_dim, self.hidden_dim),
                   self.hidden_activation]*self.num_hidden_layers
        return nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            self.hidden_activation,
            *hiddens,
            nn.Linear(self.hidden_dim, self.n_classes_))

deep_mod = TorchDeeperNeuralClassifier()


### Using both bakeoff and train data

In [15]:
bakeoff_train_size = int(0.7*len(bakeoff_dev))
print(f"Using {bakeoff_train_size} bakeoff examples for training, out of {len(bakeoff_dev)}")

Using 1652 bakeoff examples for training, out of 2361


In [16]:

bakeoff_train = bakeoff_dev.iloc[:bakeoff_train_size]
bakeoff_dev_eval = bakeoff_dev.iloc[bakeoff_train_size:]
mixed_train_df = pd.concat([sst_train, bakeoff_train])
    
mixed_train_vocab = utils.get_vocab(mixed_train_df, mincount=2)

## DynaSent

In [46]:
from datasets import load_dataset

dynasent = load_dataset("dynabench/dynasent", 'dynabench.dynasent.r1.all')
ds_train = dynasent['train'].to_pandas().rename({'gold_label': 'label'}, axis=1)

mixed_train_with_ds = pd.concat([mixed_train_df, ds_train])[mixed_train_df.columns]

Reusing dataset dynabench_dyna_sent (/Users/carolinacamassa/.cache/huggingface/datasets/dynabench___dynabench_dyna_sent/dynabench.dynasent.r1.all/1.1.0/ab89971d9ae1aacc59ed44d6855bf0e89167417257e2c2666f38e532148f2967)


  0%|          | 0/3 [00:00<?, ?it/s]

In [48]:
mixed_train_with_ds['is_subtree'] = 0.0

In [49]:
mixed_train_with_ds

,example_id,sentence,label,is_subtree
0,00001-00001,The Rock is destined to be the 21st Century 's...,positive,0.0
71,00002-00001,The gorgeously elaborate continuation of `` Th...,positive,0.0
144,00003-00001,Singer\/composer Bryan Adams contributes a sle...,positive,0.0
221,00004-00001,You 'd think by now America would have had eno...,neutral,0.0
258,00005-00001,Yet the act is still charming here .,positive,0.0
...,...,...,...,...
80483,NaN,Not to mention they killed Discount Tires prices.,neutral,0.0
80484,NaN,I have never been cussed out and told someone ...,negative,0.0
80485,NaN,From September 2010 visit:\n\nDecided to dine ...,neutral,0.0
80486,NaN,Even fast food restaraunts deserve to be revie...,neutral,0.0


## Hand-built features

In [57]:
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer
# aligning the tokenization scheme
treebank_tok = TreebankWordTokenizer()
treebank_det = TreebankWordDetokenizer()

def treebank_tokenize(s):
    return treebank_tok.tokenize(s)

def fix_treebank_tok(s):
    return treebank_det.detokenize(s.split())



In [19]:
import re
pattern_neg = r"(?:^(?:never|no|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint)$)|n't"
punctuation = '^[`.,:;!?]$'
NEG = re.compile(pattern_neg)
PUNCT = re.compile(punctuation)

def addNEG(text, tok_function):
    result = []
    startNEGATING = False
    for a in tok_function(text):
        resultNEG = NEG.findall(a.lower())

        if len(resultNEG) >0: ## IS THE WORD A NEGATION
            if startNEGATING == False: ### IS THE FIRST NEGATION
                startNEGATING = True
                result.append(a)
            else:
                result.append(a+"_NEG") ### IS AN ADDITIONAL NEGATION BEFORE THE PUNCTUATION
        else: ## IS THE WORD SOMETHING DIFFERENT THAN A NEGATION
            if startNEGATING:   #DID WE ALREADY MEET A NEGATION
                resultPUNCTUATION = PUNCT.findall(a.lower())
                if len(resultPUNCTUATION)>0:
                    startNEGATING = False
                    result.append(a)
                else:
                    result.append(a+"_NEG")
            else:
                result.append(a)
    return " ".join(result)


def unigrams_NEG(text):
    return Counter(addNEG(text.lower(), fix_treebank_tok).split())

### POS tagging

In [63]:
import spacy

nlp = spacy.load("en_core_web_sm")


def pos_tagger(text):
    out = []
    tokens = nlp(text)
    for tok in tokens:
        out.append(f"{tok.text}_{tok.pos_}")
    return out

pos_tagger(sst_train.iloc[5].sentence)

def unigrams_POS(text):
    return Counter(pos_tagger(text.lower()))

def pos_NEG_tagger(text):
    out = []
    tokens = nlp(fix_treebank_tok(text))
    for tok in tokens:
        if tok.dep_ == 'neg':
            out.append(f"{tok.text}_{tok.pos_}_NEG")
        else:
            out.append(f"{tok.text}_{tok.pos_}")
    return out

def unigrams_POS_NEG(text):
    return Counter(pos_NEG_tagger(text.lower()))



In [62]:
pos_NEG_tagger(sst_train.iloc[8].sentence)

['a_DET',
 'screenplay_NOUN',
 'more_ADV',
 'ingeniously_ADV',
 'constructed_VERB',
 'than_ADP',
 '"_PUNCT',
 'Memento_PROPN',
 '"_PUNCT']

In [ ]:

_ = sst.experiment(
    mixed_train_df,
    unigrams_POS,
    partial(fit_pipeline_classifier, mod=TorchDeeperNeuralClassifier(num_hidden_layers=3, hidden_dim=300)),
    assess_dataframes=[bakeoff_dev_eval]
    )

In [67]:
def fit_softmax_with_hyperparameter_search(X, y):
    basemod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='auto')
    cv = 5
    rescaler = TfidfTransformer()
    param_grid = {
        'model__C': [0.6, 0.8, 1.0, 2.0],
        'model__penalty': ['l1', 'l2'],
       # 'model__class_weight': ['balanced', None],
        'dim_rid__n_components': [50, 100, 200]
       }
    pipeline = Pipeline([
        ('scaler', rescaler),
        ('dim_rid', PCA()),
        ('model', basemod)])
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, pipeline, cv, param_grid)
    return bestmod

def fit_softmax(X, y):
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='auto', C=2.0, penalty='l2', class_weight='balanced')
    rescaler = TfidfTransformer()
    pipeline = Pipeline([
        ('scaler', rescaler),
        ('dim_rid',  TruncatedSVD(500)),
        ('model', mod)])
    pipeline.fit(
        X, y)
    return pipeline

### softmax classifier

In [68]:
_ = sst.experiment(
    mixed_train_df,
    unigrams_POS_NEG,
    fit_softmax_with_hyperparameter_search,
    assess_dataframes=[bakeoff_dev_eval]
    ) 

In [66]:
_ = sst.experiment(
    mixed_train_df,
    unigrams_POS_NEG,
    fit_softmax,
    assess_dataframes=[bakeoff_dev_eval]
    ) 

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

In [50]:
# with dynasent

_ = sst.experiment(
    mixed_train_with_ds,
    unigrams_POS_NEG,
    fit_softmax,
    assess_dataframes=[bakeoff_dev_eval]
    ) 

              precision    recall  f1-score   support

    negative      0.724     0.625     0.671       168
     neutral      0.733     0.769     0.750       307
    positive      0.702     0.726     0.714       234

    accuracy                          0.721       709
   macro avg      0.720     0.707     0.712       709
weighted avg      0.721     0.721     0.720       709



### unigram + NEG + torch deeper

In [ ]:
_ = sst.experiment(
    mixed_train_df,
    unigrams_POS_NEG,
    partial(fit_pipeline_classifier, mod=TorchDeeperNeuralClassifier(num_hidden_layers=3, hidden_dim=300, l2_strength=0.0001)),
    assess_dataframes=[bakeoff_dev_eval]
    )

In [ ]:

_ = sst.experiment(
    mixed_train_df,
    unigrams_NEG,
    partial(fit_pipeline_classifier, mod=deep_mod)
    )


In [ ]:
# no negation
_ = sst.experiment(
    mixed_train_df,
    unigrams_phi,
    partial(fit_pipeline_classifier, mod=deep_mod)
    )


In [ ]:
# finetuning
def fit_pipeline_classifier_with_finetuning(X, y, mod, param_grid):
    rescaler = TfidfTransformer()
    #mod = TorchShallowNeuralClassifier
    pipeline = Pipeline([
        ('scaler', rescaler),
        # We need this little bridge to go from
        # sparse matrices to dense ones:
        ('densify', utils.DenseTransformer()),
        ('model', mod)])
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, pipeline,
        param_grid=param_grid,
        cv=3)
    return pipeline

deep_mod_2 = TorchDeeperNeuralClassifier(num_hidden_layers=2, hidden_activation=nn.Tanh())

_ = sst.experiment(
    mixed_train_df,
    unigrams_NEG,
    partial(fit_pipeline_classifier, mod=deep_mod_2
            #param_grid={'model__hidden_dim': [100, 200], 'model__hidden_activation': [nn.ReLU(), nn.Tanh()]})
    ))


### bigrams + NEG + deeper neural network

In [ ]:
def bigrams_NEG(text):
    toks = addNEG(text.lower(), fix_treebank_tok).split()
    left = [utils.START_SYMBOL] + toks
    right = toks + [utils.END_SYMBOL]
    grams = list(zip(left, right))
    return Counter(grams)

_ = sst.experiment(
    mixed_train_df,
    bigrams_NEG,
    partial(fit_pipeline_classifier, mod=deep_mod)
    )
    

In [ ]:
# with SVD

_ = sst.experiment(
    mixed_train_df,
    bigrams_NEG,
    partial(fit_pipeline_classifier, mod=deep_mod, dim_red=TruncatedSVD(300))
    )
    

### Yelp embedding (PMI and LSA/SVD) + NN

In [ ]:
VSMDATA_HOME = os.path.join('data', 'vsmdata')
import vsm

yelp20 = pd.read_csv(
    os.path.join(VSMDATA_HOME, 'yelp_window20-flat.csv.gz'), index_col=0)
yelp20_ppmi = vsm.pmi(yelp20)
yelp20_ppmi_svd = vsm.lsa(yelp20_ppmi, k=300)
yelp_lookup = dict(zip(yelp20_ppmi_svd.index, yelp20_ppmi_svd.values))

def vsm_phi(text, lookup, np_func=np.mean):
    """Represent `text` as a combination of the vector of its words.

    Parameters
    ----------
    text : str

    lookup : dict
        From words to vectors.

    np_func : function (default: np.sum)
        A numpy matrix operation that can be applied columnwise,
        like `np.mean`, `np.sum`, or `np.prod`. The requirement is that
        the function take `axis=0` as one of its arguments (to ensure
        columnwise combination) and that it return a vector of a
        fixed length, no matter what the size of the text is.

    Returns
    -------
    np.array, dimension `X.shape[1]`

    """
    allvecs = np.array([lookup[w] for w in text.split() if w in lookup])
    if len(allvecs) == 0:
        dim = len(next(iter(lookup.values())))
        feats = np.zeros(dim)
    else:
        feats = np_func(allvecs, axis=0)
    return feats




In [ ]:

_ = sst.experiment(
    mixed_train_df,
    partial(vsm_phi, lookup=yelp_lookup),
    partial(fit_pipeline_classifier, mod=deep_mod),
    vectorize=False,
    assess_dataframes=[bakeoff_dev_eval, sst_dev]
    )

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# SYSTEM DESCRIPTION
# DATA USED: I added in both the Dynasent training set and 70% of the bakeoff dev dataset (the rest was used to assess performance)
# PREPROCESSING: I used spacy to add POS and negation markings to each token, then counted unigrams.
# The dataset was then rescaled with TfIdf. I also tried dim. reduction (PCA with 200, 300 dimensions) but the score worsened.
# MODEL: I used a simple LogisticRegression model.
# The model parameters were finetuned with 5-fold cross validation.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    import sys, re, sst, spacy
    from sklearn.decomposition import TruncatedSVD, PCA, NMF
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import TfidfTransformer
    from functools import partial
    !{sys.executable} -m spacy download en_core_web_sm
    
    nlp = spacy.load("en_core_web_sm")

    # POS and negation marking

    def pos_NEG_tagger(text):
        out = []
        tokens = nlp(text)
        for tok in tokens:
            if tok.dep_ == 'neg':
                out.append(f"{tok.text}_{tok.pos_}_NEG")
            else:
                out.append(f"{tok.text}_{tok.pos_}")
        return out

    def unigrams_POS_NEG(text):
        return Counter(pos_NEG_tagger(text))

    # model
    def fit_softmax(X, y):
        mod = LogisticRegression(
            fit_intercept=True,
            solver='liblinear',
            multi_class='auto', C=2.0, penalty='l2', class_weight='balanced')
        rescaler = TfidfTransformer()
        pipeline = Pipeline([
            ('scaler', rescaler),
        # ('dim_rid', )),
            ('model', mod)])
        pipeline.fit(
            X, y)
        return pipeline

    

# STOP COMMENT: Please do not remove this comment.

## Bakeoff [1 point]

As we said above, the bakeoff evaluation data is the official SST test-set release and a new test set derived from the same sources and labeling methods as for `bakeoff_dev`.

For this bakeoff, you'll evaluate your original system from the above homework problem on these test sets. Our metric will be the mean of the macro-F1 values, which weights both datasets equally despite their differing sizes.

The central requirement for your system is that you have define a `predict_one` method for it that maps a text (str) directly to a label prediction – one of 'positive', 'negative', 'neutral'. If you used `sst.experiment` with `vectorize=True`, then the following function (for `softmax_experiment`) will be easy to adapt – you probably just need to change the variable `softmax_experiment` to the variable for your experiment output.

In [ ]:
def predict_one_softmax(text):
    # Singleton list of feature dicts:
    feats = [softmax_experiment['phi'](text)]
    # Vectorize to get a feature matrix:
    X = softmax_experiment['train_dataset']['vectorizer'].transform(feats)
    # Standard sklearn `predict` step:
    preds = softmax_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

If you used an RNN like the one we demoed above, then featurization is a bit more straightforward:

In [ ]:
def predict_one_rnn(text):
    # List of tokenized examples:
    X = [rnn_experiment['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = rnn_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

The following function is used to create the bakeoff submission file. Its arguments are your `predict_one` function and an output filename (str).

In [ ]:
def create_bakeoff_submission(
        predict_one_func,
        output_filename='cs224u-sentiment-bakeoff-entry.csv'):

    bakeoff_test = sst.bakeoff_test_reader(SST_HOME)
    sst_test = sst.test_reader(SST_HOME)
    bakeoff_test['dataset'] = 'bakeoff'
    sst_test['dataset'] = 'sst3'
    df = pd.concat((bakeoff_test, sst_test))

    df['prediction'] = df['sentence'].apply(predict_one_func)

    df.to_csv(output_filename, index=None)

Thus, for example, the following will create a bake-off entry based on `predict_one_softmax`:

In [ ]:
# This check ensure that the following code only runs on the local environment only.
# The following call will not be run on the autograder environment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    create_bakeoff_submission(predict_one_softmax)

This creates a file `cs224u-sentiment-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in our bakeoff prediction file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission instructions

Review and follow the [Homework and bake-off code: Formatting guide](hw_formatting_guide.ipynb).
Please do not change the filename as described below.

Submit the following files to Gradescope:

- `hw_sentiment.ipynb` (this notebook)
- `cs224u-sentiment-bakeoff-entry.csv` (bake-off output)


# Todo


- align sst and bakeoff (sst has different tokenization, see 1st sst notebook)